### 数据加载

In [8]:
import numpy as np
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.neighbors import KNeighborsRegressor#回归，平均值，计算房价中位数

In [2]:
data=load_boston()

/Users/liuriyilang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California

In [3]:
data

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [4]:
X=data.data
y=data.target
display(X.shape,y.shape)

(506, 13)

(506,)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
X_train.shape

(404, 13)

# 交叉验证-超参数筛选

In [9]:
from sklearn.model_selection import  GridSearchCV#网格搜索  cv交叉验证能够进行批量筛选
#k值，weight权重，p表示距离的度量：欧氏距离哈曼顿距离
k=[1,3,5,7,9,15,17,23,30]
weights=['uniform','distance']
p=[1,2]
#对以上三个超参数进行自动组合
params=dict(n_neighbors=k,weights=weights,p=p)
estimator=KNeighborsRegressor()
#cv分为5折 scoring选择的是均方误差，去sk官网找名字
gCV=GridSearchCV(estimator,params,cv=5,scoring='neg_mean_squared_error')
gCV.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 15, 17, 23, 30],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

In [10]:
gCV.best_params_
#最合适的参数

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [12]:
gCV.best_score_
#最好的分熟均方误差

-33.2971448976826

In [13]:
best_knn=gCV.best_estimator_
#最好的模型
best_knn

KNeighborsRegressor(p=1, weights='distance')

In [15]:
y_=best_knn.predict(X_test)
print(y_[:20])

[24.73786364 31.72249009 15.3507154  27.30501395 17.2018207  25.80550606
 19.77102868 15.86582451 19.94556132 20.51957611 23.34136189 19.56304115
 12.34755581 21.52977457 23.20852325 22.26416486 19.81820761 14.24896371
 35.11804765 11.62004094]


In [16]:
print(y_test[:20])

[23.6 32.4 13.6 22.8 16.1 20.  17.8 14.  19.6 16.8 21.5 18.9  7.  21.2
 18.5 29.8 18.8 10.2 50.  14.1]


In [17]:
from sklearn.metrics import mean_squared_error#均方误差

In [19]:
mean_squared_error(y_test,y_)
#均方误差越小越好

20.998924877624457

In [20]:
((y_test-y_)**2).mean()

20.998924877624457